#GENERAR KARDEX


In [ ]:
# @title Cargar Datos Kardex { form-width: "2px", display-mode: "form" }
!pip install tabulate > /dev/null
!pip install xlsxwriter > /dev/null
!pip install openpyxl > /dev/null
!pip install secure-smtplib > /dev/null
!pip install python-magic > /dev/null

# @title
#Librerias
import numpy as np
from pandas import ExcelWriter as ex
import warnings
import numpy as np
from openpyxl.styles import Border, Side
import pandas as pd
from google.colab import files
from tables import Column
import matplotlib.pyplot as plt
from tabulate import tabulate
from prettytable import PrettyTable
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.styles import Font, Alignment, PatternFill
from prettytable import PrettyTable
import pathlib
import warnings
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from datetime import datetime
from google.colab import drive



# Disable all warnings (not recommended)
warnings.filterwarnings("ignore")

#Leer Documentos

# Montar Google Drive sin imprimir mensajes
drive.mount('/content/drive')




Activos_AF = pd.read_excel('/content/drive/MyDrive/Proyectos Bodega/kardex/Documentos/Activos AF.xlsx', dtype={'Código de barras':object,'Número de serie':object,'Sub-Ubicación':object,'Responsable':object,'Ubicación':object})
Activos_GAF = pd.read_excel('/content/drive/MyDrive/Proyectos Bodega/kardex/Documentos/Activos GAF.xlsx',dtype={'Código de barras':object,'Número de serie':object,'Sub-Ubicación':object,'Responsable':object,'Ubicación':object})
Ubicaciones = pd.read_excel('/content/drive/MyDrive/Proyectos Bodega/kardex/Documentos/Ubicaciones.xlsx',dtype={'Ubicación':object,'Nombre':object})
Sububicaciones = pd.read_excel('/content/drive/MyDrive/Proyectos Bodega/kardex/Documentos/Sububicaciones.xlsx', dtype={'Código de Sub-Ubicación': object,'Placa':object,'Id. de sububicación':object})
Responsables = pd.read_excel('/content/drive/MyDrive/Proyectos Bodega/kardex/Documentos/Responsables.xlsx',dtype={'Número de personal':object})

#Eliminar columnas
Activos_AF=Activos_AF.drop(['Capa de registro','Tipo','Grupo de activos fijos','Libro'], axis=1)
Activos_GAF=Activos_GAF.drop(['Capa de registro','Tipo','Grupo de activos fijos','Libro'],axis=1)
Responsables=Responsables.drop(['Conocido como'],axis=1)

# Función para convertir datos en fecha, pesos y texto

def convert_data(df):
    df['Fecha de adquisición'] = pd.to_datetime(df['Fecha de adquisición'], format='%d/%m/%y')
    df['Fecha de cancelación'] = pd.to_datetime(df['Fecha de cancelación'], format='%d/%m/%y')
    columns_to_convert = ['Precio de adquisición', 'Código de barras', 'Número de serie', 'TAG', 'Sub-Ubicación']
    df[columns_to_convert] = df[columns_to_convert].astype(str)
    df['Precio de adquisición'] = df['Precio de adquisición'].str.replace('.', ',')

# Aplicar funciones de conversión

convert_data(Activos_AF)
convert_data(Activos_GAF)
Ubicaciones['Ubicación'] = Ubicaciones['Ubicación'].astype(str)
Sububicaciones['Código de Sub-Ubicación'] = Sububicaciones['Código de Sub-Ubicación'].astype(str)
Responsables['Número de personal'] = Responsables['Número de personal'].astype(str)


# Aplicar búsqueda para traer nombre de ubicaciones, sububicaciones y responsables
Activos_AF.insert(7,'Nombre de Ubicación',pd.Series([],dtype=str))
Activos_AF.insert(9,'Nombre de Sub_Ubicación',pd.Series([],dtype=str))
Activos_AF.insert(10,'Placa',pd.Series([],dtype=str))
Activos_AF.insert(11,'Id',pd.Series([],dtype=str))
Activos_AF.insert(13,'Nombre de Responsable',pd.Series([],dtype=str))

Activos_GAF.insert(7,'Nombre de Ubicación',pd.Series([],dtype=str))
Activos_GAF.insert(9,'Nombre de Sub_Ubicación',pd.Series([],dtype=str))
Activos_GAF.insert(10,'Placa',pd.Series([],dtype=str))
Activos_GAF.insert(11,'Id',pd.Series([],dtype=str))
Activos_GAF.insert(13,'Nombre de Responsable',pd.Series([],dtype=str))

#Aplicar "buscarv" para traer nombre ub, sub ubicación y resposable
ListadodeUbicaciones = Ubicaciones.set_index(['Ubicación'])
ListadodeSububicaciones = Sububicaciones.set_index(['Código de Sub-Ubicación'])
ListadodeResponsables = Responsables.set_index(['Número de personal'])
#---------------
Activos_AF=Activos_AF.merge(ListadodeUbicaciones , how='left', left_on= Activos_AF['Ubicación'], right_on='Ubicación',right_index=True)
Activos_AF.loc[:,'Nombre de Ubicación']=Activos_AF.loc[:,'Nombre_y']
Activos_AF = Activos_AF.drop(["Nombre_y"], axis=1)

Activos_AF=Activos_AF.merge(ListadodeSububicaciones , how='left', left_on= Activos_AF['Sub-Ubicación'], right_on='Código de Sub-Ubicación',right_index=True)
Activos_AF.loc[:,'Nombre de Sub_Ubicación']=Activos_AF.loc[:,"Nombre de sububicación"]
Activos_AF = Activos_AF.drop(["Nombre de sububicación"], axis=1)
Activos_AF = Activos_AF.drop(["Tipo de sububicación"], axis=1)
Activos_AF = Activos_AF.drop(["Placa_y"], axis=1)
Activos_AF = Activos_AF.drop(["Id. de sububicación"], axis=1)

Activos_AF=Activos_AF.merge(ListadodeSububicaciones , how='left', left_on= Activos_AF['Sub-Ubicación'], right_on='Código de Sub-Ubicación',right_index=True)
Activos_AF.loc[:,'Placa_x']=Activos_AF.loc[:,'Placa']
Activos_AF = Activos_AF.drop(["Nombre de sububicación"], axis=1)
Activos_AF = Activos_AF.drop(["Tipo de sububicación"], axis=1)
Activos_AF = Activos_AF.drop(["Placa"], axis=1)
Activos_AF = Activos_AF.drop(["Id. de sububicación"], axis=1)

Activos_AF=Activos_AF.merge(ListadodeSububicaciones , how='left', left_on= Activos_AF['Sub-Ubicación'], right_on='Código de Sub-Ubicación',right_index=True)
Activos_AF.loc[:,'Id']=Activos_AF.loc[:,'Id. de sububicación']
Activos_AF = Activos_AF.drop(["Nombre de sububicación"], axis=1)
Activos_AF = Activos_AF.drop(["Tipo de sububicación"], axis=1)
Activos_AF = Activos_AF.drop(["Placa"], axis=1)
Activos_AF = Activos_AF.drop(["Id. de sububicación"], axis=1)

Activos_AF=Activos_AF.merge(ListadodeResponsables , how='left', left_on= Activos_AF['Responsable'], right_on='Número de personal',right_index=True)
Activos_AF.loc[:,'Nombre de Responsable']=Activos_AF.loc[:,'Nombre']
Activos_AF = Activos_AF.drop(["Nombre"], axis=1)
#----
Activos_GAF=Activos_GAF.merge(ListadodeUbicaciones , how='left', left_on= Activos_GAF['Ubicación'], right_on='Ubicación',right_index=True)
Activos_GAF.loc[:,'Nombre de Ubicación']=Activos_GAF.loc[:,'Nombre_y']
Activos_GAF = Activos_GAF.drop(["Nombre_y"], axis=1)

Activos_GAF=Activos_GAF.merge(ListadodeSububicaciones , how='left', left_on= Activos_GAF['Sub-Ubicación'], right_on='Código de Sub-Ubicación',right_index=True)
Activos_GAF.loc[:,'Nombre de Sub_Ubicación']=Activos_GAF.loc[:,'Nombre de sububicación']
Activos_GAF = Activos_GAF.drop(["Nombre de sububicación"], axis=1)
Activos_GAF = Activos_GAF.drop(["Tipo de sububicación"], axis=1)
Activos_GAF = Activos_GAF.drop(["Placa_y"], axis=1)
Activos_GAF = Activos_GAF.drop(["Id. de sububicación"], axis=1)

Activos_GAF=Activos_GAF.merge(ListadodeSububicaciones , how='left', left_on= Activos_GAF['Sub-Ubicación'], right_on='Código de Sub-Ubicación',right_index=True)
Activos_GAF.loc[:,'Placa_x']=Activos_GAF.loc[:,'Placa']
Activos_GAF = Activos_GAF.drop(["Nombre de sububicación"], axis=1)
Activos_GAF = Activos_GAF.drop(["Tipo de sububicación"], axis=1)
Activos_GAF = Activos_GAF.drop(["Placa"], axis=1)
Activos_GAF = Activos_GAF.drop(["Id. de sububicación"], axis=1)

Activos_GAF=Activos_GAF.merge(ListadodeSububicaciones , how='left', left_on= Activos_GAF['Sub-Ubicación'], right_on='Código de Sub-Ubicación',right_index=True)
Activos_GAF.loc[:,'Id']=Activos_GAF.loc[:,'Id. de sububicación']
Activos_GAF = Activos_GAF.drop(["Nombre de sububicación"], axis=1)
Activos_GAF = Activos_GAF.drop(["Tipo de sububicación"], axis=1)
Activos_GAF = Activos_GAF.drop(["Placa"], axis=1)
Activos_GAF = Activos_GAF.drop(["Id. de sububicación"], axis=1)

Activos_GAF=Activos_GAF.merge(ListadodeResponsables , how='left', left_on= Activos_GAF['Responsable'], right_on='Número de personal',right_index=True)
Activos_GAF.loc[:,'Nombre de Responsable']=Activos_GAF.loc[:,'Nombre']
Activos_GAF = Activos_GAF.drop(["Nombre"], axis=1)


# Filtrar por estado del activo

Activos_Abierto = Activos_AF[Activos_AF['Estado'] == 'Abierto']
Activos_Cerrado = Activos_AF[Activos_AF['Estado'] == 'Cerrado']
Activos_no_Adquirido = Activos_AF[Activos_AF['Estado'] == 'No adquirido todavía']

Activos_Bajas = Activos_AF[Activos_AF['Estado'] == 'Dado de baja']
Activos_Vendido = Activos_AF[Activos_AF['Estado'] == 'Vendido']

Activos_GAF_Abierto= Activos_GAF[Activos_GAF['Estado'] == 'Abierto']
Activos_GAF_NO= Activos_GAF[Activos_GAF['Estado'] == 'No adquirido todavía']

Activos_Cerrado = Activos_Cerrado.append(Activos_no_Adquirido, ignore_index=True)
Activos_Abierto = Activos_Abierto.append(Activos_Cerrado, ignore_index=True)
Activos_Abierto=Activos_Abierto.drop(['Fecha de cancelación'], axis=1)
Activos_Abierto = Activos_Abierto.replace('nan', None)

Activos_Bajas = Activos_Bajas.append(Activos_Vendido, ignore_index=True)
Activos_Bajas = Activos_Bajas.replace('nan', None)


Activos_GAF_Abierto = Activos_GAF_Abierto.append(Activos_GAF_NO, ignore_index=True)
Activos_GAF_NO = Activos_GAF_NO.append(Activos_GAF_Abierto, ignore_index=True)
Activos_GAF_Abierto=Activos_GAF_Abierto.drop(['Fecha de cancelación'], axis=1)
Activos_GAF_Abierto = Activos_GAF_Abierto.replace('nan', None)

kardex_path = '/content/drive/MyDrive/Proyectos Bodega/kardex/Kardex/Kardex.xlsx'
with pd.ExcelWriter(kardex_path, engine='xlsxwriter') as writer:
    # Escribir los DataFrames en el archivo Excel
    Activos_Abierto.to_excel(writer, sheet_name='Kardex', index=False)
    Activos_Bajas.to_excel(writer, sheet_name='Bajas', index=False)
    Activos_GAF_Abierto.to_excel(writer, sheet_name='Activos 1512', index=False)
    Ubicaciones.to_excel(writer, sheet_name='Ubicación', index=False)
    Sububicaciones.to_excel(writer, sheet_name='Sub-ubicación', index=False)
    Responsables.to_excel(writer, sheet_name='Responsable', index=False)

# Cargar el archivo Excel y obtener el objeto workbook
book = load_workbook(kardex_path)

# Establecer un estilo para el formato deseado
header_style = Font(name='Calibri', size=11, color="00FFFFFF")  # Blanco
cell_alignment = Alignment(horizontal='center')
fill = PatternFill(start_color="002060", end_color="002060", fill_type="solid")  # Azul oscuro
# Iterar sobre todas las hojas del archivo
for sheet_name in book.sheetnames:
    sheet = book[sheet_name]

    # Aplicar formato a los encabezados
    for header_cell in sheet[1]:
        header_cell.font = header_style
        header_cell.alignment = cell_alignment
        header_cell.fill = fill
    for column_cells in sheet.columns:
            max_length = 0
            column = [cell for cell in column_cells]
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            sheet.column_dimensions[column[0].column_letter].width = adjusted_width

# Función para reemplazar espacios en blanco por guiones
def reemplazar_espacios_con_guiones(sheet):
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        for cell in row:
            if cell.value == " " or cell.value is None:
                cell.value = "-"

# Agregar bordes y reemplazar espacios en blanco con guiones en todas las celdas en cada hoja
for sheet_name in book.sheetnames:
    sheet = book[sheet_name]
    border_style = Border(left=Side(border_style='thin', color='000000'),
                         right=Side(border_style='thin', color='000000'),
                         top=Side(border_style='thin', color='000000'),
                         bottom=Side(border_style='thin', color='000000'))

    # Aplicar bordes
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        for cell in row:
            cell.border = border_style

    # Reemplazar espacios en blanco con guiones
    reemplazar_espacios_con_guiones(sheet)

for sheet_name in book.sheetnames:
    sheet = book[sheet_name]
    sheet.freeze_panes = 'A2'  # Fija la primera fila, excluyendo la celda A
# Guardar el archivo Excel con el formato aplicado
book.save(kardex_path)

# Desactivar temporalmente las advertencias
with warnings.catch_warnings():
    warnings.simplefilter("ignore")


print("ARCHIVO CREADO Y GUARDADO CON FORMATO")


ARCHIVO CREADO Y GUARDADO CON FORMATO


In [ ]:
# @title Enviada al correo { form-width: "1px", display-mode: "form" }


def enviar_correo(destinatario, asunto, cuerpo, adjunto_path):
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    smtp_usuario = "aprendizinventarios@rbsas.co"
    smtp_clave = "123456Jsn#1"
    # Obtener la fecha actual y formatearla como parte del nombre del archivo
    fecha_actual = datetime.now().strftime("%d-%m-%Y")
    asunto_con_fecha = f"Kardex-{fecha_actual}"
    nombre_adjunto = f"Kardex-{fecha_actual}.xlsx"

    msg = MIMEMultipart()
    msg['From'] = smtp_usuario
    msg['To'] = destinatario
    msg['Subject'] = asunto_con_fecha  # Asunto con la fecha actual
    msg.attach(MIMEText(cuerpo, 'plain'))



    with open(adjunto_path, "rb") as adjunto:
        part = MIMEApplication(adjunto.read(), Name=nombre_adjunto)
        part['Content-Disposition'] = f'attachment; filename="{nombre_adjunto}"'
        msg.attach(part)

    try:
        servidor = smtplib.SMTP(smtp_server, smtp_port)
        servidor.starttls()
        servidor.login(smtp_usuario, smtp_clave)
        servidor.sendmail(smtp_usuario, destinatario, msg.as_string())
        servidor.quit()
        print("Correo electrónico enviado exitosamente.")
    except Exception as e:
        print(f"Error al enviar el correo electrónico: {str(e)}")

# Destinatario del correo electrónico
correo_destino = 'chblanco@rbsas.co'  # Reemplaza con la dirección de correo a la que deseas enviar el archivo

# Asunto y cuerpo del correo electrónico
correo_asunto = 'Kardex'
correo_cuerpo = '''Cordial Saludo,
Compañeros
Comparto kardex del día'''

# Llamada a la función para enviar el correo electrónico con el archivo adjunto
enviar_correo(correo_destino, correo_asunto, correo_cuerpo, kardex_path)


In [ ]:
# @title Consulta de activos { form-width: "1px", display-mode: "form" }

# Concatenar DataFrames
book = pd.concat([Activos_Abierto, Activos_Bajas, Activos_GAF, Ubicaciones, Sububicaciones, Responsables], ignore_index=True)

# Funciones de búsqueda y visualización
def mostrar_menu():
    print("\n=== Menú de Búsqueda ===")
    print("1. Búsqueda por serie")
    print("2. Búsqueda por número de activo fijo")
    print("3. Búsqueda por código de barras")
    print("4. Búsqueda por nombre de Ubicación")
    print("5. Búsqueda por placa bus")
    print("0. Salir")
    print("========================")

def realizar_busqueda(opcion_busqueda):
    try:
        print("========================")
        cantidad = int(input(f'\n¿Cuántas {opcion_busqueda} desea consultar? '))
        resultados = pd.DataFrame()
        print("========================")

        for i in range(cantidad):
            print("========================")
            print(f"\nConsulta {i + 1} de {cantidad}")
            print("========================")
            resultado = None

            if opcion_busqueda == '1':
                resultado = busqueda_por_serie()
            elif opcion_busqueda == '2':
                resultado = busqueda_por_numero_activo()
            elif opcion_busqueda == '3':
                resultado = busqueda_por_codigo_de_barras()
            elif opcion_busqueda == '4':
                resultado = busqueda_por_Nombre_de_ubicacion()
            elif opcion_busqueda == '5':
                resultado = busqueda_por_Placa_Bus()

            if resultado is not None and not resultado.empty:
                resultados = pd.concat([resultados, resultado], ignore_index=True)
            else:
                print("========================")
                print("No se encontraron resultados para esta búsqueda.")
                print("========================")

        return resultados
    except ValueError:
        print("========================")
        print("Error: Ingrese un número válido.")
        print("========================")
        return pd.DataFrame()
def validar_opcion(opcion):
    try:
        opcion = int(opcion)
        if 0 <= opcion <= 5:
            return str(opcion)
        else:
            print("========================")
            print("Error: Ingrese un número válido entre 0 y 5.")
            print("========================")
            return None
    except ValueError:
        print("========================")
        print("Error: Ingrese un número entero.")
        print("========================")
        return None

def busqueda_por_serie():
    print("========================")
    serie = input('Escriba el serial a consultar: ')
    print("========================")
    return book[book['Número de serie'] == serie]

def busqueda_por_numero_activo():
    numero_af = input('Escriba el número de activo fijo a consultar: ')
    return book[book['Número de activo fijo'] == numero_af]

def busqueda_por_codigo_de_barras():
    codigo_de_barras = input('Escriba el número del código de barras a consultar: ')
    return book[book['Código de barras'] == codigo_de_barras]

def busqueda_por_Nombre_de_ubicacion():
    nombre_de_ubicacion = input('Escriba el nombre de ubicación a consultar: ')
    return book[book['Nombre de Ubicación'] == nombre_de_ubicacion]

def busqueda_por_Placa_Bus():
    numero_de_placa_bus = input('Escriba el número de la placa bus a consultar: ')
    return book[book['Placa_x'] == numero_de_placa_bus]

def guardar_resultado_excel(resultado):
    print("========================")
    respuesta = input('¿Desea guardar esta consulta en un archivo Excel? (si/no): ').lower()
    print("========================")

    if respuesta == 'si':
        ruta_excel = '/content/drive/MyDrive/Proyectos Bodega/kardex/Busqueda kardex/Resultado_Busqueda.xlsx'

        # Cargamos el archivo Excel existente o creamos uno nuevo
        try:
            book_excel = load_workbook(ruta_excel)
        except FileNotFoundError:
            book_excel = Workbook()

        # Eliminamos el contenido existente de la hoja de cálculo
        if 'Resultado_Busqueda' in book_excel.sheetnames:
            sheet = book_excel['Resultado_Busqueda']
            for row in sheet.iter_rows(min_row=2, max_col=sheet.max_column, max_row=sheet.max_row):
                for cell in row:
                    cell.value = None

        # Guardamos la nueva consulta en la hoja de cálculo
        with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
            writer.book = book_excel
            resultado.to_excel(writer, 'Resultado_Busqueda', index=False)
            sheet = writer.sheets['Resultado_Busqueda']
            formato_excel(sheet)

        path = pathlib.Path(ruta_excel)
        path.absolute().with_suffix('.xlsx').as_posix()
        print("========================")
        print('Consulta guardada en', path)
        print("")
        print("========================")

def formato_excel(sheet):
    header_style = Font(name='Calibri', size=11, color="00FFFFFF")
    cell_alignment = Alignment(horizontal='center')
    fill = PatternFill(start_color="002060", end_color="002060", fill_type="solid")

    for header_cell in sheet[1]:
        header_cell.font = header_style
        header_cell.alignment = cell_alignment
        header_cell.fill = fill

    for column_cells in sheet.columns:
        max_length = 0
        column = [cell for cell in column_cells]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column[0].column_letter].width = adjusted_width

if __name__ == "__main__":
    while True:

        mostrar_menu()
        opcion = input("Seleccione una opción (0 para salir): ")
        print("========================")

        if opcion == '0':
            break

        opcion = validar_opcion(opcion)
        resultado = realizar_busqueda(opcion)

        if resultado is not None and not resultado.empty:
            mostrar_resultado(resultado)
        else:
            print("========================")
            print("No se encontraron resultados para la búsqueda.")
            print("========================")



